ゼロからはじめるPython(12) 過去の気象情報をもとに運動会の開催日を選んでみよう(その1) - 気象データの入手と簡単な統計 | TECH+  
https://news.mynavi.jp/article/zeropython-12/

※ 記事だと配列に分解して合計を出しているけど、Pandasの方でMultiIndexを利用したほうが簡単なのでそちらを利用した

In [140]:
import pandas as pd

In [141]:
df = pd.read_csv("data.csv", encoding="SHIFT_JIS") \
      .rename(columns={'年月日': 'date', '降水量の合計(mm)': 'rainfall_sum', '天気概況(昼：06時〜18時)': 'wheater'}) \
      .set_index('date')

# Set index
df.index = pd.to_datetime(df.index)

# Set multi index
df = df.set_index([df.index.year, df.index.quarter, df.index.month, df.index.weekday, df.index.day, df.index])
df.index.names = ['year', 'quarter', 'month', 'weekday', 'day', 'date']

df

rainfall_sum wheater
year quarter month weekday day date                            
2011 1       1     5       1   2011-01-01           0.0   晴後一時曇
                   6       2   2011-01-02           0.0   晴一時薄曇
                   0       3   2011-01-03           0.0    曇一時晴
                   1       4   2011-01-04           0.0       晴
                   2       5   2011-01-05           0.0  晴後時々薄曇
...                                                 ...     ...
2020 4       12    0       28  2020-12-28           0.0   曇後時々晴
                   1       29  2020-12-29           0.0    曇時々晴
                   2       30  2020-12-30           8.5  雨後曇時々晴
                   3       31  2020-12-31           0.0   晴後一時曇
2021 1       1     4       1   2021-01-01           0.0       晴

[3654 rows x 2 columns]

In [142]:
daily_sum_df = df.sum(level=['month', 'day'])
daily_sum_df[daily_sum_df['rainfall_sum'] == 0.0]

rainfall_sum
month day              
1     1             0.0
      2             0.0
      3             0.0
      4             0.0
      5             0.0
      10            0.0
      11            0.0
      12            0.0
      13            0.0
      16            0.0
      19            0.0
      25            0.0
2     3             0.0
      24            0.0
3     12            0.0
4     16            0.0
5     5             0.0
6     4             0.0
12    21            0.0

In [143]:
print(daily_sum_df[daily_sum_df['rainfall_sum'] == 0.0].index)

MultiIndex([( 1,  1),
            ( 1,  2),
            ( 1,  3),
            ( 1,  4),
            ( 1,  5),
            ( 1, 10),
            ( 1, 11),
            ( 1, 12),
            ( 1, 13),
            ( 1, 16),
            ( 1, 19),
            ( 1, 25),
            ( 2,  3),
            ( 2, 24),
            ( 3, 12),
            ( 4, 16),
            ( 5,  5),
            ( 6,  4),
            (12, 21)],
           names=['month', 'day'])


In [146]:
indexes = daily_sum_df[daily_sum_df['rainfall_sum'] == 0.0].index
df.xs(indexes[0], level=['month', 'day'], drop_level=False)

,,,,,,rainfall_sum,wheater
year,quarter,month,weekday,day,date,,
2011,1,1,5,1,2011-01-01,0.0,晴後一時曇
2012,1,1,6,1,2012-01-01,0.0,曇
2013,1,1,1,1,2013-01-01,0.0,晴時々薄曇
2014,1,1,2,1,2014-01-01,0.0,快晴
2015,1,1,3,1,2015-01-01,0.0,曇時々雪一時晴、あられを伴う
2016,1,1,4,1,2016-01-01,0.0,快晴
2017,1,1,6,1,2017-01-01,0.0,快晴
2018,1,1,0,1,2018-01-01,0.0,晴
2019,1,1,1,1,2019-01-01,0.0,晴一時曇


In [162]:
result = df[0:0]

for index in indexes.values:
  values = df.xs(index, level=['month', 'day'], drop_level=False)
  result = result.append(values)

result = result.drop(columns='rainfall_sum')
result

,,,,,,wheater
year,quarter,month,weekday,day,date,
2011,1,1,5,1,2011-01-01,晴後一時曇
2012,1,1,6,1,2012-01-01,曇
2013,1,1,1,1,2013-01-01,晴時々薄曇
2014,1,1,2,1,2014-01-01,快晴
2015,1,1,3,1,2015-01-01,曇時々雪一時晴、あられを伴う
...,...,...,...,...,...,...
2016,4,12,2,21,2016-12-21,晴後薄曇
2017,4,12,3,21,2017-12-21,晴一時曇
2018,4,12,4,21,2018-12-21,快晴
